# JSON examples and exercise
****
+ get familiar with packages for dealing with JSON
+ study examples with JSON strings and files 
+ work on exercise to be completed and submitted 
****
+ reference: http://pandas.pydata.org/pandas-docs/stable/io.html#io-json-reader
+ data source: http://jsonstudio.com/resources/
****

In [3]:
import pandas as pd

## imports for Python, Pandas

In [4]:
import json
from pandas.io.json import json_normalize

## JSON example, with string

+ demonstrates creation of normalized dataframes (tables) from nested json string
+ source: http://pandas.pydata.org/pandas-docs/stable/io.html#normalization

In [11]:
# define json string
data = [{'state': 'Florida', 
         'shortname': 'FL',
         'info': {'governor': 'Rick Scott'},
         'counties': [{'name': 'Dade', 'population': 12345},
                      {'name': 'Broward', 'population': 40000},
                      {'name': 'Palm Beach', 'population': 60000}]},
        {'state': 'Ohio',
         'shortname': 'OH',
         'info': {'governor': 'John Kasich'},
         'counties': [{'name': 'Summit', 'population': 1234},
                      {'name': 'Cuyahoga', 'population': 1337}]}]

In [5]:
pd.DataFrame(data)

In [6]:
# use normalization to create tables from nested element
json_normalize(data, 'counties')

In [7]:
# further populate tables created from nested element
json_normalize(data, 'counties', ['state', 'shortname', ['info', 'governor']])

****
## JSON example, with file

+ demonstrates reading in a json file as a string and as a table
+ uses small sample file containing data about projects funded by the World Bank 
+ data source: http://jsonstudio.com/resources/

In [8]:
# load json as string from source
json.load((open('data/world_bank_projects_less.json')))

In [15]:
# load as Pandas dataframe
sample_json_df = pd.read_json('data/world_bank_projects_less.json')
sample_json_df

,_id,approvalfy,board_approval_month,boardapprovaldate,borrower,closingdate,country_namecode,countrycode,countryname,countryshortname,...,sectorcode,source,status,supplementprojectflg,theme1,theme_namecode,themecode,totalamt,totalcommamt,url
0,{'$oid': '52b213b38594d8a2be17c780'},1999,November,2013-11-12T00:00:00Z,FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA,2018-07-07T00:00:00Z,Federal Democratic Republic of Ethiopia!$!ET,ET,Federal Democratic Republic of Ethiopia,Ethiopia,...,"ET,BS,ES,EP",IBRD,Active,N,"{'Name': 'Education for all', 'Percent': 100}","[{'name': 'Education for all', 'code': '65'}]",65,130000000,130000000,http://www.worldbank.org/projects/P129828/ethi...
1,{'$oid': '52b213b38594d8a2be17c781'},2015,November,2013-11-04T00:00:00Z,GOVERNMENT OF TUNISIA,NaN,Republic of Tunisia!$!TN,TN,Republic of Tunisia,Tunisia,...,"BZ,BS",IBRD,Active,N,"{'Name': 'Other economic management', 'Percent...","[{'name': 'Other economic management', 'code':...","54,24",0,4700000,http://www.worldbank.org/projects/P144674?lang=en


****
## JSON exercise

Using data in file 'data/world_bank_projects.json' and the techniques demonstrated above,
1. Find the 10 countries with most projects
2. Find the top 10 major project themes (using column 'mjtheme_namecode')
3. In 2. above you will notice that some entries have only the code and the name is missing. Create a dataframe with the missing names filled in.

In [1]:
import pandas as pd
import numpy as np
import json
from pandas.io.json import json_normalize

 #set options to display all columns of df
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)

In [2]:
#1. Find the 10 countries with the most projects

 # load data 
wb = pd.read_json('data/world_bank_projects.json')

 # group by country w/ #projects, sort descending, and remove non-country entries
countries = wb.groupby('countryshortname').size()
csort = countries.sort_values(ascending=False).head(11) 
csort.drop(['Africa'])

countryshortname
Indonesia             19
China                 19
Vietnam               17
India                 16
Yemen, Republic of    13
Nepal                 12
Bangladesh            12
Morocco               12
Mozambique            11
Pakistan               9
dtype: int64

In [3]:
#2. Find the top 10 major project themes (using column 'mjtheme_namecode')

 # load data as json file and normalize around project name and id
theme_json = json.load((open('data/world_bank_projects.json')))
theme = json_normalize(theme_json, 'mjtheme_namecode', ['id'])

In [4]:
 # sort by code, fill blank project names, check with crosstab
theme_sort = theme.sort_values(by = 'code')
theme_fill = theme_sort.replace('', np.nan,regex=True).fillna(method = 'pad')
pd.crosstab(index = theme_fill['code'], columns = theme_fill['name'])

name,Economic management,Environment and natural resources management,Financial and private sector development,Human development,Public sector governance,Rule of law,Rural development,Social dev/gender/inclusion,Social protection and risk management,Trade and integration,Urban development
code,,,,,,,,,,,
1,38,0,0,0,0,0,0,0,0,0,0
10,0,0,0,0,0,0,216,0,0,0,0
11,0,250,0,0,0,0,0,0,0,0,0
2,0,0,0,0,199,0,0,0,0,0,0
3,0,0,0,0,0,15,0,0,0,0,0
4,0,0,146,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,77,0
6,0,0,0,0,0,0,0,0,168,0,0
7,0,0,0,0,0,0,0,130,0,0,0


In [5]:
 #group and sort by project theme to get top 10
theme_grouped = theme_fill.groupby('name').size()
theme_grouped.sort_values(ascending=False).head(10) 

name
Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
dtype: int64

In [6]:
#3. In 2. above you will notice that some entries have only the code and the name is missing. 
#   Create a dataframe with the missing names filled in.

# Filling in of missing values was performed in question 2, so convert result into df here and add column heading
theme_df = pd.DataFrame(theme_grouped)
theme_df.columns = ['frequency']
theme_df

,frequency
name,
Economic management,38
Environment and natural resources management,250
Financial and private sector development,146
Human development,210
Public sector governance,199
Rule of law,15
Rural development,216
Social dev/gender/inclusion,130
Social protection and risk management,168
